# **IMPORT MODEL**

In [1]:
from keras.models import load_model
import pickle

# Define the paths
model_path = 'model2.keras'
tokenizer_path = 'tokenizer.pickle'
label_encoder_path = 'label_encoder.pickle'

model = load_model('model2.keras')

# Load the tokenizer
with open(tokenizer_path, 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load the label encoder
with open(label_encoder_path, 'rb') as handle:
    label_encoder = pickle.load(handle)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# **PDF INPUT**

In [4]:
!pip install PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 55.1 MB/s eta 0:00:00


In [7]:
import fitz
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
import re

PATH_TO_PDF = '515729.pdf'

def remove_non_characters(text):
    # regular expression pattern
    pattern = r'[^a-zA-ZçÇğĞıİöÖşŞüÜ\s]'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)

    # List for the PDF  Text
    text_list = []

    references_index = 0
    search_words = ["kaynaklar", "literatür listesi", "kaynaklar dizini", "kaynakça", "referanslar", "bibliyografya"]

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        page_text = page.get_text()
        text_list.append(page_text)

    # ALgorithm to find References Page
    for page_num in range(len(pdf_document)-1, 0, -1):
      page = pdf_document.load_page(page_num)
      page_text = page.get_text()

      lines = page_text.split('\n')
      lines = [line for line in lines if line.strip()]
      for line in range(min(len(lines),5)):
          if remove_non_characters(lines[line]).strip().lower() in search_words:
              references_index = page_num+1
              pdf_document.close()
              return text_list, references_index

    pdf_document.close()
    return text_list, references_index

new_texts, references_index = extract_text_from_pdf(PATH_TO_PDF)
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded_sequences = pad_sequences(new_sequences, maxlen=200, padding='post', truncating='post')

predictions = tf.nn.sigmoid(model.predict(new_padded_sequences)).numpy()
predicted_labels = (predictions > 0.5).astype(int)
predicted_labels = label_encoder.inverse_transform(predicted_labels.flatten())

output_list = []

# Calculating length
length = len(predicted_labels)

# If not found make all pages after last 1 as 1
if references_index == 0 or references_index < int(length * 0.5):
    #update ekler(Appendix)
    i = len(predicted_labels) - 1
    while(predicted_labels[i] != 1):
      predicted_labels[i] = 1
      i -= 1
else:
    # Found
    end = references_index - 1
    found = True
    for i in range(end, len(predicted_labels)):
        predicted_labels[i] = 1

for i in range(len(predicted_labels)):
  if predicted_labels[i] == 1:
    output_list.append(i+1)

print(f"Toplam sayfa sayısı: {len(predicted_labels)}")
print(output_list)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 251ms/step
Toplam sayfa sayısı: 100
[5, 94, 95, 96, 97, 98, 99, 100]
